In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_atac_cage_early as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_atac_cage_early as training_utils


from scipy import stats

2023-03-06 17:37:54.130009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 17:37:54.290851: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-06 17:37:54.290888: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-06 17:37:55.131514: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-7')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2023-03-06 17:38:00.745202: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-06 17:38:00.759111: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-06 17:38:00.759185: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2023-03-06 17:38:01.112196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 17:38:01.632796: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:447] Started server with target: grpc://localhost:39414
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-7


INFO:tensorflow:Initializing the TPU system: node-7


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
g = tf.random.Generator.from_seed(datetime.now().timestamp())
with strategy.scope():
    
    data_tr,data_val,data_val_TSS = training_utils.return_distributed_iterators("gs://picard-testing-176520/genformer_atac_rampage_globalacc_conv_rpgc_5prime_65k",
                                                                                "gs://picard-testing-176520/genformer_atac_rampage_globalacc_conv_rpgc_TSS_5prime_65k",
                                                                               GLOBAL_BATCH_SIZE,
                                                                               65536,
                                                                               10,
                                                                                16384,
                                                                               512,
                                                                               100,
                                                                               128,
                                                                               8,
                                                                               50,
                                                                               strategy,
                                                                               options,
                                                                                True,
                                                                                0.25,
                                                                                True,
                                                                                True,
                                                                                True,
                                                                               g)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


created train + val


In [12]:
with strategy.scope():
    #inits = training_utils.get_initializers_enformer_performer("gs://picard-testing-176520/enformer_performer_pretrain_atac_mean/models/enformer_performer_pretrain_atac_mean_EP_full_dataset__196k_load_init-True_freeze-False_LR1-1e-06_LR2-0.0001_T-4_F-1536_D-0.4_2023-02-24_16:27:50/iteration_16",
    #                                                           4,
     #                                                          stable_variant=False)
    #inits = training_utils.get_initializers_enformer_conv("/home/jupyter/dev/BE_CD69_paper_2022/enformer_fine_tuning/checkpoint/sonnet_weights")
    inits = training_utils.get_initializers_enformer_conv("gs://picard-testing-176520/enformer_performer/models/enformer_performer_230303_E-P-_5313_enformer_196k_load_init-False_freeze-False_LR1-0.0001_LR2-0.0001_T-6_F-1024_K-relu_kernel_transformation/iteration_21",
                                                              False,
                                                              6)
    model = aformer.aformer(kernel_transformation = 'relu_kernel_transformation',
                            dropout_rate = 0.10,
                            pointwise_dropout_rate=0.15,
                            input_length = 65536,
                            output_length = 512,
                            final_output_length = 312,
                             num_heads = 8,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             num_transformer_layers = 2,
                             norm=True,
                            BN_momentum=0.90,
                             max_seq_length = 512,
                             use_rot_emb = True,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=True,
                            stable_variant=False,
                             inits=inits,
                            freeze_conv_layers=False,
                            predict_masked_atac_bool=True,
                            learnable_PE=True,
                            global_acc_size=128,
                            filter_list_seq=[512,640,768,896,1024,1024],
                            inits_type='enformer_conv')

In [13]:
with strategy.scope():
    optimizer1 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-06,
        epsilon= 1.0e-14,
        weight_decay= 1.0e-08,
        rectify=True
    )
    optimizer2 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-04,
        epsilon= 1.0e-14,
        weight_decay= 1.0e-08,
        rectify=True
    )
    
    optimizers_in=optimizer1,optimizer2
    

In [14]:
with strategy.scope():
    metric_dict = {}
    train_step_masked_atac,train_step, \
        val_step_masked_atac,val_step, \
            val_step_TSS_masked_atac, val_step_TSS, \
                build_step, metric_dict = training_utils.return_train_val_functions(model,
                                                                                    250,
                                                                                    200,
                                                                                     200,
                                                                                      optimizers_in,
                                                                                      strategy,
                                                                                      metric_dict,
                                                                                      GLOBAL_BATCH_SIZE,
                                                                                    10.0,
                                                                                      0.5,
                                                                                    'poisson')
    


In [ ]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 10):
        start = time.time()
        if epoch_i == 1:
            build_step(data_val)
            total_params = 0
            for k in model.trainable_variables:
                var = k.values[0]
                total_params += tf.size(var)
            print('total params: ' + str(total_params))


        train_step_masked_atac(data_tr)
        #train_step(data_tr)

        print('hg_train_loss: ' + str(metric_dict['train_loss'].result().numpy()))
        
        val_step_masked_atac(data_val)
        #val_step(data_val)

        val_losses.append(metric_dict['val_loss'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['val_loss'].result().numpy()))
        print('hg_val_loss_CAGE: ' + str(metric_dict['val_loss_CAGE'].result().numpy()))
        print('hg_val_loss_ATAC: ' + str(metric_dict['val_loss_ATAC'].result().numpy()))
        print('hg_val_cage_pearson: ' + str(metric_dict['CAGE_PearsonR'].result()['PearsonR'].numpy()))
        print('hg_val_cage_R2: ' + str(metric_dict['CAGE_R2'].result()['R2'].numpy()))
        
        print('hg_val_atac_pearson: ' + str(metric_dict['ATAC_PearsonR'].result()['PearsonR'].numpy()))
        print('hg_val_atac_R2: ' + str(metric_dict['ATAC_R2'].result()['R2'].numpy()))
        
        
        #print('hg_val_atac_baseline_pearson: ' + str(metric_dict['ATAC_PearsonR_baseline'].result()['PearsonR'].numpy()))
        #print('hg_val_atac_baseline_R2: ' + str(metric_dict['ATAC_R2_baseline'].result()['R2'].numpy()))
        
        val_step_TSS_masked_atac(data_val_TSS)
        #val_step_TSS(data_val_TSS)
        
        y_trues = metric_dict['corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['corr_stats'].result()['gene_map'].numpy()


        figures, corrs_overall = training_utils.make_plots(y_trues,
                                           y_preds, 
                                           cell_types, 
                                           gene_map)
        
        cell_spec,gene_spec = corrs_overall
        
        print('cell spec corr: ' + str(cell_spec))
        print('gene_spec_corr:' + str(gene_spec))
        

        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))
        
        for key, item in metric_dict.items():
            item.reset_state()
        

total params: tf.Tensor(52269202, shape=(), dtype=int32)
hg_train_loss: 0.059835494
hg_val_loss: 0.03489942
hg_val_loss_CAGE: 0.052928988
hg_val_loss_ATAC: -0.01802957
hg_val_cage_pearson: [0.19464584]
hg_val_cage_R2: [0.00042999]
hg_val_atac_pearson: 0.3535583
hg_val_atac_R2: -0.20411837
cell spec corr: 0.5143402710964984
gene_spec_corr:0.13158912387777844
completed epoch 1
duration(mins): 4.234614996115367
patience counter at: 0
hg_train_loss: -0.18868966
hg_val_loss: -0.08231039
hg_val_loss_CAGE: 0.047559153
hg_val_loss_ATAC: -0.12986955
hg_val_cage_pearson: [0.1837289]
hg_val_cage_R2: [0.00126147]
hg_val_atac_pearson: 0.5336082
hg_val_atac_R2: 0.15287465
cell spec corr: 0.6138315473343459
gene_spec_corr:0.13319314365272072
completed epoch 2
duration(mins): 3.5744242548942564
patience counter at: 0
hg_train_loss: -0.22066444
hg_val_loss: -0.030830897
hg_val_loss_CAGE: 0.11360992
hg_val_loss_ATAC: -0.14444074
hg_val_cage_pearson: [0.02260528]
hg_val_cage_R2: [-0.00082994]
hg_val_atac

In [19]:
model.conv_tower.layers[0].trainable_variables

[TPUDistributedVariable:{
   0: <tf.Variable 'sync_batch_normalization_22/gamma:0' shape=(512,) dtype=float32, numpy=
 array([0.18252954, 0.16885765, 0.1430062 , 0.15492319, 0.15358064,
        0.14575574, 0.15441139, 0.15865992, 0.16979796, 0.15770608,
        0.15222842, 0.174316  , 0.15495321, 0.15752323, 0.15718684,
        0.16940689, 0.19433723, 0.17388585, 0.16788991, 0.14734316,
        0.15070128, 0.1539612 , 0.18382192, 0.16899614, 0.1468947 ,
        0.16287586, 0.17419256, 0.16284177, 0.16058554, 0.13209653,
        0.17577825, 0.16612528, 0.18302089, 0.13121966, 0.14351934,
        0.15712595, 0.1755188 , 0.16986053, 0.1750114 , 0.17195667,
        0.15087828, 0.13841745, 0.18529451, 0.17122662, 0.17316104,
        0.20323592, 0.16725667, 0.19863355, 0.15331101, 0.177306  ,
        0.15289979, 0.16298616, 0.1646904 , 0.16524577, 0.14763585,
        0.18002714, 0.13688968, 0.1611788 , 0.17496385, 0.16493832,
        0.13434742, 0.18654408, 0.17001636, 0.1713432 , 0.14806679

In [20]:
checkpoint_path="gs://picard-testing-176520/enformer_performer/models/enformer_performer_230303_E-P-_5313_enformer_196k_load_init-False_freeze-False_LR1-0.0001_LR2-0.0001_T-6_F-1024_K-relu_kernel_transformation/iteration_21"
inside_checkpoint=tf.train.list_variables(tf.train.latest_checkpoint(checkpoint_path))
reader = tf.train.load_checkpoint(checkpoint_path)

In [23]:
reader.get_tensor('conv_tower/layer_with_weights-0/layer_with_weights-2/_logit_linear/kernel/.ATTRIBUTES/VARIABLE_VALUE')


array([[ 3.17296863e-01,  1.02415616e-02,  1.01509353e-03, ...,
        -1.79714598e-02,  3.12023163e-02, -1.65626989e-03],
       [ 2.25408878e-02,  3.25529397e-01,  7.82050472e-03, ...,
         9.97012854e-03,  4.73789461e-02,  3.01890112e-02],
       [-2.65965872e-02,  1.48091679e-02,  3.24585885e-01, ...,
         8.78635049e-03, -2.22406592e-02,  6.16294937e-03],
       ...,
       [-4.82196081e-03,  2.42240983e-03, -1.37088373e-02, ...,
         3.38781238e-01, -3.77864726e-02, -8.51365679e-04],
       [ 1.66197270e-02,  2.68676970e-02, -1.61379408e-02, ...,
        -6.98795635e-03,  3.52226675e-01,  2.40519736e-02],
       [ 7.85722304e-03,  3.77347507e-02,  2.63770016e-06, ...,
        -1.42793665e-02,  2.41297930e-02,  3.44751388e-01]], dtype=float32)

In [25]:
model.final_pointwise_conv.layers[0].momentum = 0.70

In [26]:
model.final_pointwise_conv.layers[0].momentum

0.7

In [44]:
def deserialize_tr(serialized_example,
                   input_length,
                   max_shift,
                   output_length,
                   crop_size,
                   output_res,
                   predict_masked_atac_bool,
                   atac_mask_dropout,
                   g):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'atac': tf.io.FixedLenFeature([], tf.string),
        'cage': tf.io.FixedLenFeature([], tf.string),
        'processed_gene_token': tf.io.FixedLenFeature([], tf.string)
    }
    ### stochastic sequence shift and gaussian noise


    rev_comp = tf.math.round(g.uniform([], 0, 1))

    shift = g.uniform(shape=(),
                      minval=0,
                      maxval=max_shift,
                      dtype=tf.int32)

    for k in range(max_shift):
        if k == shift:
            interval_end = input_length + k
            seq_shift = k
        else:
            seq_shift=0
    
    input_seq_length = input_length + max_shift


    data = tf.io.parse_example(serialized_example, feature_map)
    #sequence = one_hot(tf.strings.substr(data['sequence'],
    #                             seq_shift,input_length))
    
    atac = tf.ensure_shape(tf.io.parse_tensor(data['atac'],
                                              out_type=tf.float32),
                           [output_length,1])
    #atac = atac + tf.math.abs(g.normal(atac.shape,
    #                                           mean=0.0,
    #                                           stddev=0.05,
    #                                           dtype=tf.float32))
    
    masked_atac=tf.nn.experimental.stateless_dropout(atac, 
                                                     rate=atac_mask_dropout, 
                                                     seed=[0,seq_shift]) / (1. / (1.0 -atac_mask_dropout))
    
    cage = tf.ensure_shape(tf.io.parse_tensor(data['cage'],
                                              out_type=tf.float32),
                           [output_length - 2*crop_size,1])

        
    gene_token= tf.io.parse_tensor(data['processed_gene_token'],
                                   out_type=tf.int32)
        
    if predict_masked_atac_bool:
        atac_out = tf.slice(atac,
                            [crop_size,0],
                            [output_length-2*crop_size,-1])
        target = tf.concat([atac_out,cage],axis=1)
        
    if predict_masked_atac_bool:
    
        return {#'sequence': tf.ensure_shape(sequence,
                #                            [input_length,4]),
                'atac': tf.ensure_shape(masked_atac,
                                          [output_length,1]),
                'gene_token':gene_token,
                'target': tf.ensure_shape(target,
                                          [output_length-crop_size*2,2])}
    else:
        return {#'sequence': tf.ensure_shape(sequence,
                #                            [input_length,4]),
                'atac': tf.ensure_shape(atac,
                                          [output_length,1]),
                'gene_token':gene_token}
                #'target': tf.ensure_shape(cage,
                #                          [output_length-crop_size*2,1])}
    
    
g = tf.random.Generator.from_seed(datetime.now().timestamp())
dataset = tf.data.TFRecordDataset("gs://picard-testing-176520/genformer_atac_rampage_globalacc_conv_TSS/valid/HG_A375.tfr",
                                  compression_type='ZLIB',
                                  num_parallel_reads=4)

dataset = dataset.map(lambda record: deserialize_tr(record,
                                                    196618,
                                                    10,
                                                    1536,
                                                    320,
                                                    128,
                                                    False,
                                                    0.0,
                                                    g),
                      deterministic=True,
                      num_parallel_calls=4)
        
    
dataset1 = tf.data.TFRecordDataset("gs://picard-testing-176520/enformer_atac_rampage_paired_basenji_tss/valid/HG_A375.tfr",
                                  compression_type='ZLIB',
                                  num_parallel_reads=4)

dataset1 = dataset1.map(lambda record: deserialize_tr(record,
                                                    196618,
                                                    10,
                                                    1536,
                                                    320,
                                                    128,
                                                    False,
                                                    0.0,
                                                    g),
                      deterministic=True,
                      num_parallel_calls=4)
        

In [49]:
test = iter(dataset)

In [55]:
next(test)

{'atac': <tf.Tensor: shape=(1536, 1), dtype=float32, numpy=
 array([[0.45809558],
        [0.47703928],
        [1.2499237 ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]], dtype=float32)>,
 'gene_token': <tf.Tensor: shape=(), dtype=int32, numpy=6870>}

In [51]:
test = iter(dataset1)

In [56]:
next(test)

{'atac': <tf.Tensor: shape=(1536, 1), dtype=float32, numpy=
 array([[0.        ],
        [0.        ],
        [0.72191423],
        ...,
        [5.7844844 ],
        [1.4600831 ],
        [0.        ]], dtype=float32)>,
 'gene_token': <tf.Tensor: shape=(), dtype=int32, numpy=15816>}

In [4]:
checkpoint_path = "gs://picard-testing-176520/paired_rampage_atac/genformer/models/aformer_baseline_GENFORMER_glob_accTrue_atac-True_mask-True_load-True_frz-False_LR1-5e-06_LR2-0.0001_T-6_D-0.05_2023-03-03_01:51:20/iteration_30"
inside_checkpoint=tf.train.list_variables(tf.train.latest_checkpoint(checkpoint_path))
reader = tf.train.load_checkpoint(checkpoint_path)

In [6]:
inside_checkpoint

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/gamma/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/moving_mean/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/moving_variance/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-1/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [5, 768, 768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-1/_layer/layer_with_weights-0/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-1/_layer/layer_with_weig